In [1]:
! unzip /content/drive/MyDrive/torch/DIP/blurred.zip

Archive:  /content/drive/MyDrive/torch/DIP/blurred.zip
   creating: blurred/
  inflating: __MACOSX/._blurred      
  inflating: blurred/348.png         
  inflating: __MACOSX/blurred/._348.png  
  inflating: blurred/412.png         
  inflating: __MACOSX/blurred/._412.png  
  inflating: blurred/374.png         
  inflating: __MACOSX/blurred/._374.png  
  inflating: blurred/360.png         
  inflating: __MACOSX/blurred/._360.png  
  inflating: blurred/406.png         
  inflating: __MACOSX/blurred/._406.png  
  inflating: blurred/1145.png        
  inflating: __MACOSX/blurred/._1145.png  
  inflating: blurred/1151.png        
  inflating: __MACOSX/blurred/._1151.png  
  inflating: blurred/638.png         
  inflating: __MACOSX/blurred/._638.png  
  inflating: blurred/176.png         
  inflating: __MACOSX/blurred/._176.png  
  inflating: blurred/88.png          
  inflating: __MACOSX/blurred/._88.png  
  inflating: blurred/610.png         
  inflating: __MACOSX/blurred/._610.png  
  in

In [2]:
! unzip /content/drive/MyDrive/torch/DIP/sharp.zip

Archive:  /content/drive/MyDrive/torch/DIP/sharp.zip
   creating: sharp/
  inflating: __MACOSX/._sharp        
  inflating: sharp/348.png           
  inflating: __MACOSX/sharp/._348.png  
  inflating: sharp/412.png           
  inflating: __MACOSX/sharp/._412.png  
  inflating: sharp/374.png           
  inflating: __MACOSX/sharp/._374.png  
  inflating: sharp/360.png           
  inflating: __MACOSX/sharp/._360.png  
  inflating: sharp/406.png           
  inflating: __MACOSX/sharp/._406.png  
  inflating: sharp/1145.png          
  inflating: __MACOSX/sharp/._1145.png  
  inflating: sharp/1151.png          
  inflating: __MACOSX/sharp/._1151.png  
  inflating: sharp/638.png           
  inflating: __MACOSX/sharp/._638.png  
  inflating: sharp/176.png           
  inflating: __MACOSX/sharp/._176.png  
  inflating: sharp/88.png            
  inflating: __MACOSX/sharp/._88.png  
  inflating: sharp/610.png           
  inflating: __MACOSX/sharp/._610.png  
  inflating: sharp/604.png    

In [3]:
! unzip /content/drive/MyDrive/torch/DIP/blurred_noise.zip

Archive:  /content/drive/MyDrive/torch/DIP/blurred_noise.zip
   creating: blurred_noise/
  inflating: blurred_noise/141_sp.png  
  inflating: blurred_noise/610_sp.png  
  inflating: blurred_noise/251_poisson.png  
  inflating: blurred_noise/710_sp.png  
  inflating: blurred_noise/542_gaussian.png  
  inflating: blurred_noise/701_poisson.png  
  inflating: blurred_noise/517_gaussian.png  
  inflating: blurred_noise/949_gaussian.png  
  inflating: blurred_noise/805_sp.png  
  inflating: blurred_noise/1109_sp.png  
  inflating: blurred_noise/187_poisson.png  
  inflating: blurred_noise/905_sp.png  
  inflating: blurred_noise/299_gaussian.png  
  inflating: blurred_noise/548_poisson.png  
  inflating: blurred_noise/266_gaussian.png  
  inflating: blurred_noise/1009_sp.png  
  inflating: blurred_noise/233_gaussian.png  
  inflating: blurred_noise/1085_gaussian.png  
  inflating: blurred_noise/344_gaussian.png  
  inflating: blurred_noise/311_gaussian.png  
  inflating: blurred_noise/878_sp.

In [4]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
from os.path import join
from PIL import Image
from tqdm import tqdm
import torch.optim as optim

In [5]:
class Noise(Dataset):
    def __init__(self, root, gt_root, transform=None):
        """ Intialize the dataset """
        self.filenames = []
        self.root = root
        self.gt_root = gt_root
        self.transform = transform

        # read filenames
        for i in range(1,1152):
            blur, gau, sp, poi = join(gt_root,str(i)+'.png'), join(root,str(i)+'_gaussian.png'), join(root,str(i)+'_sp.png'), join(root,str(i)+'_poisson.png')
            blur, gau, sp, poi = Image.open(blur), Image.open(gau), Image.open(sp), Image.open(poi)
            if self.transform is not None:
                blur = self.transform(blur)
                gau = self.transform(gau)
                sp = self.transform(sp)
                poi = self.transform(poi)
            self.filenames.append([blur,gau,sp,poi])

        self.len = len(self.filenames)

    def __getitem__(self, index):
        """ Get a sample from the dataset """
        blur, gau, sp, poi = self.filenames[index]
        
        return blur, gau, sp, poi

    def __len__(self):
        """ Total number of samples in the dataset """
        return self.len

In [8]:
def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [6]:
from torchvision.transforms.transforms import RandomCrop
# Create the dataset. 
# transforms.ToTensor() automatically converts PIL images to
trans = transforms.Compose([
    #transforms.Resize(128),
    transforms.CenterCrop(256),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

NoiseSet = Noise(root='/content/blurred_noise',gt_root='/content/blurred',transform=trans)
train_set, val_set = torch.utils.data.random_split(NoiseSet, [900, 251])
train_loader = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_set, batch_size=8, num_workers=0)

# Denoise AE

In [7]:
def test(model):
    criterion = nn.MSELoss()
    model.eval()  # Important: set evaluation mode
    gau_loss, sp_loss, poi_loss = 0,0,0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for origin, gau, sp, poi in val_loader:
            origin, gau, sp, poi = origin.cuda(), gau.cuda(), sp.cuda(), poi.cuda()
            output = model(gau)
            gau_loss += criterion(output, origin).item()
            output = model(sp)
            sp_loss += criterion(output, origin).item()
            output = model(poi)
            poi_loss += criterion(output, origin).item()

    # l = len(val_set)
    # gau_loss /= l
    # sp_loss /= l
    # poi_loss /= l
    print('\nValidation set: Gaussian loss: {:.4f}, Salt&Pepper loss: {:.4f}, Poisson loss: {:.4f}\n'.format(gau_loss, sp_loss, poi_loss))

In [9]:
from torch.autograd import Variable
import torchvision.models as models

# def conv_block(in_channels, out_channels):
#     bn = nn.BatchNorm2d(out_channels)
#     nn.init.uniform_(bn.weight) # for pytorch 1.2 or later
#     return nn.Sequential(
#         nn.Conv2d(in_channels, out_channels, 3, padding=1),
#         bn,
#         nn.ReLU(),
#         nn.MaxPool2d(2)
#     )

# class Convnet(nn.Module):

#     def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
#         super().__init__()
#         self.encoder = nn.Sequential(
#             conv_block(x_dim, hid_dim),
#             conv_block(hid_dim, hid_dim),
#             conv_block(hid_dim, hid_dim),
#             conv_block(hid_dim, z_dim),
#         )
#     def forward(self, x):
#         x = self.encoder(x)
#         return x.view(x.size(0), -1)

class Encoder(nn.Module):
    def __init__(self, nc, nef, nz, isize, device):
        super(Encoder, self).__init__()

        # Device
        self.device = device

        # Encoder: (nc, isize, isize) -> (nef*8, isize//16, isize//16)
        self.encoder = nn.Sequential(
            nn.Conv2d(nc, nef, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef),

            nn.Conv2d(nef, nef*2, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef*2),

            nn.Conv2d(nef*2, nef*4, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef*4),

            nn.Conv2d(nef*4, nef*8, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef*8)
        )

        # Map the encoded feature map to the latent vector of mean, (log)variance
        out_size = isize // 16
        self.mean = nn.Linear(nef*8*out_size*out_size, nz)
        self.logvar = nn.Linear(nef*8*out_size*out_size, nz)

    def reparametrize(self, mean, logvar):
        std = logvar.mul(0.5).exp_()
        multi_norm = torch.FloatTensor(std.size()).normal_().to(self.device)
        multi_norm = Variable(multi_norm)
        return multi_norm.mul(std).add_(mean)

    def forward(self, inputs):
        # Batch size
        batch_size = inputs.size(0)
        # Encoded feature map
        hidden = self.encoder(inputs)
        # Reshape
        hidden = hidden.view(batch_size, -1)
        # Calculate mean and (log)variance
        mean, logvar = self.mean(hidden), self.logvar(hidden)
        # Sample
        latent_z = self.reparametrize(mean, logvar)

        return latent_z, mean, logvar

class Decoder(nn.Module):
    def __init__(self, nc, ndf, nz, isize):
        super(Decoder, self).__init__()

        # Map the latent vector to the feature map space
        self.ndf = ndf
        self.out_size = isize // 16
        self.decoder_dense = nn.Sequential(
            nn.Linear(nz, ndf*8*self.out_size*self.out_size),
            nn.ReLU(True)
        )
        # Decoder: (ndf*8, isize//16, isize//16) -> (nc, isize, isize)
        self.decoder_conv = nn.Sequential(
            nn.UpsamplingNearest2d(scale_factor=2),
            nn.Conv2d(ndf*8, ndf*4, 3, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(ndf*4, 1.e-3),

            nn.UpsamplingNearest2d(scale_factor=2),
            nn.Conv2d(ndf*4, ndf*2, 3, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(ndf*2, 1.e-3),
        
            nn.UpsamplingNearest2d(scale_factor=2),
            nn.Conv2d(ndf*2, ndf, 3, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(ndf, 1.e-3),

            nn.UpsamplingNearest2d(scale_factor=2),
            nn.Conv2d(ndf, nc, 3, padding=1)
        )

    def forward(self, input):
        batch_size = input.size(0)
        hidden = self.decoder_dense(input).view(
            batch_size, self.ndf*8, self.out_size, self.out_size)
        output = self.decoder_conv(hidden)
        return output

class DAE(nn.Module):
    def __init__(self, nc=3, ndf=32, nef=32, nz=100, isize=256, device=torch.device("cuda:0"), is_train=True):
        super(DAE, self).__init__()

        self.nz = nz
        # Encoder
        self.encoder = Encoder(nc=nc, nef=nef, nz=nz, isize=isize, device=device)
        # Decoder
        self.decoder = Decoder(nc=nc, ndf=ndf, nz=nz, isize=isize)

        if is_train == False:
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = False

    def forward(self, x):
        latent_z, mean, logvar = self.encoder(x)
        rec_x = self.decoder(latent_z)
        return rec_x
    
    def encode(self, x):
        latent_z, _, _ = self.encoder(x)
        return latent_z

    def decode(self, z):
        return self.decoder(z)

In [98]:
model = DAE().cuda()
load_checkpoint('/content/drive/MyDrive/torch/DIP/DenoiseAE_new.pth', model, optimizer)
model.train()  # set training mode

model loaded from /content/drive/MyDrive/torch/DIP/DenoiseAE_new.pth


DAE(
  (encoder): Encoder(
    (encoder): Sequential(
      (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (4): LeakyReLU(negative_slope=0.2, inplace=True)
      (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (7): LeakyReLU(negative_slope=0.2, inplace=True)
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (10): LeakyReLU(negative_slope=0.2, inplace=True)
      (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (mean): Linear(in_features=65536, out_

In [105]:
epoch = 1000
optimizer = optim.Adam(model.parameters(), lr=1e-7)
#lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
criterion = nn.MSELoss()
# load_checkpoint('/content/drive/MyDrive/torch/DIP/DenoiseAE_0103.pth', model, optimizer)

In [157]:
mn = 0.3
for ep in range(epoch):
    g,s,p,now = 0,0,0,0
    for batch_idx, (origin, gau, sp, poi) in enumerate(train_loader):
        origin, gau, sp, poi = origin.cuda(), gau.cuda(), sp.cuda(), poi.cuda()
        output = model(gau)
        gau_loss = criterion(output, origin)
        output = model(sp)
        sp_loss = criterion(output, origin)
        output = model(poi)
        poi_loss = criterion(output, origin)
        loss = (gau_loss + sp_loss + poi_loss)/3
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
            
        g += gau_loss.item()
        s += sp_loss.item()
        p += poi_loss.item()

    #lr_scheduler.step()
    # l = len(train_set)
    # g /= l
    # s /= l
    # p /= l
    now = (g+s+p)/3
    if now < mn:
        mn = now
        save_checkpoint('/content/drive/MyDrive/torch/DIP/DenoiseAE_0105.pth', model, optimizer)

    print('Train Epoch: {} Gaussian loss: {:.4f}, Salt&Pepper loss: {:.4f}, Poisson loss: {:.4f}\n'.format(ep, g, s, p))
    test(model)


RuntimeError: ignored

In [66]:
save_checkpoint('/content/drive/MyDrive/torch/DIP/DenoiseAE_0104.pth', model, optimizer)


model saved to /content/drive/MyDrive/torch/DIP/DenoiseAE_0104.pth


# UNet

In [145]:
"""
The UNet model credit to jakeoung
https://github.com/jakeoung/Unet_pytorch/blob/master/model.py
"""

import torch
from torch import nn
import torch.nn.functional as F

def add_conv_stage(dim_in, dim_out, kernel_size=3, stride=1, padding=1, bias=True, useBN=False):
  if useBN:
    return nn.Sequential(
      nn.Conv2d(dim_in, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
      nn.BatchNorm2d(dim_out),
      nn.LeakyReLU(0.1),
      nn.Conv2d(dim_out, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
      nn.BatchNorm2d(dim_out),
      nn.LeakyReLU(0.1)
    )
  else:
    return nn.Sequential(
      nn.Conv2d(dim_in, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
      nn.ReLU(),
      nn.Conv2d(dim_out, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
      nn.ReLU()
    )

def add_merge_stage(ch_coarse, ch_fine, in_coarse, in_fine, upsample):
  conv = nn.ConvTranspose2d(ch_coarse, ch_fine, 4, 2, 1, bias=False)
  torch.cat(conv, in_fine)

  return nn.Sequential(
    nn.ConvTranspose2d(ch_coarse, ch_fine, 4, 2, 1, bias=False)
  )
  upsample(in_coarse)

def upsample(ch_coarse, ch_fine):
  return nn.Sequential(
    nn.ConvTranspose2d(ch_coarse, ch_fine, 4, 2, 1, bias=False),
    nn.ReLU()
  )

class Net(nn.Module):
  def __init__(self, useBN=False):
    super(Net, self).__init__()

    self.conv1   = add_conv_stage(1, 32, useBN=useBN)
    self.conv2   = add_conv_stage(32, 64, useBN=useBN)
    self.conv3   = add_conv_stage(64, 128, useBN=useBN)
    self.conv4   = add_conv_stage(128, 256, useBN=useBN)
    self.conv5   = add_conv_stage(256, 512, useBN=useBN)

    self.conv4m = add_conv_stage(512, 256, useBN=useBN)
    self.conv3m = add_conv_stage(256, 128, useBN=useBN)
    self.conv2m = add_conv_stage(128,  64, useBN=useBN)
    self.conv1m = add_conv_stage( 64,  32, useBN=useBN)

    self.conv0  = nn.Sequential(
        nn.Conv2d(32, 1, 3, 1, 1),
        nn.Sigmoid()
    )

    self.max_pool = nn.MaxPool2d(2)

    self.upsample54 = upsample(512, 256)
    self.upsample43 = upsample(256, 128)
    self.upsample32 = upsample(128,  64)
    self.upsample21 = upsample(64 ,  32)

    ## weight initialization
    for m in self.modules():
      if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        if m.bias is not None:
          m.bias.data.zero_()


  def forward(self, x):
    conv1_out = self.conv1(x)
    #return self.upsample21(conv1_out)
    conv2_out = self.conv2(self.max_pool(conv1_out))
    conv3_out = self.conv3(self.max_pool(conv2_out))
    conv4_out = self.conv4(self.max_pool(conv3_out))
    conv5_out = self.conv5(self.max_pool(conv4_out))

    conv5m_out = torch.cat((self.upsample54(conv5_out), conv4_out), 1)
    conv4m_out = self.conv4m(conv5m_out)

    conv4m_out_ = torch.cat((self.upsample43(conv4m_out), conv3_out), 1)
    conv3m_out = self.conv3m(conv4m_out_)

    conv3m_out_ = torch.cat((self.upsample32(conv3m_out), conv2_out), 1)
    conv2m_out = self.conv2m(conv3m_out_)

    conv2m_out_ = torch.cat((self.upsample21(conv2m_out), conv1_out), 1)
    conv1m_out = self.conv1m(conv2m_out_)

    conv0_out = self.conv0(conv1m_out)

    return conv0_out

In [146]:
device = 'cuda'
model = Net().to(device) # try decreasing the depth value if there is a memory error


In [ ]:
from torchsummary import summary
summary(model,(1,512,512))

In [ ]:
model.train()  # set training mode

In [152]:
epoch = 100
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
#lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
criterion = nn.MSELoss()
# load_checkpoint('/content/drive/MyDrive/torch/DIP/DenoiseAE_0103.pth', model, optimizer)

In [153]:
def model_RGB(img_in):
    img_R = model(img_in[:,0,:,:].unsqueeze(1))
    img_G = model(img_in[:,1,:,:].unsqueeze(1))
    img_B = model(img_in[:,2,:,:].unsqueeze(1))
    img_out = torch.cat((img_R,img_G,img_B),dim=1)
    return img_out

In [158]:
def test(model):
    criterion = nn.MSELoss()
    model.eval()  # Important: set evaluation mode
    gau_loss, sp_loss, poi_loss = 0,0,0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for origin, gau, sp, poi in val_loader:
            origin, gau, sp, poi = origin.cuda(), gau.cuda(), sp.cuda(), poi.cuda()
            output = model_RGB(gau)
            gau_loss += criterion(output, origin).item()
            output = model_RGB(sp)
            sp_loss += criterion(output, origin).item()
            output = model_RGB(poi)
            poi_loss += criterion(output, origin).item()

    # l = len(val_set)
    # gau_loss /= l
    # sp_loss /= l
    # poi_loss /= l
    print('\nValidation set: Gaussian loss: {:.4f}, Salt&Pepper loss: {:.4f}, Poisson loss: {:.4f}\n'.format(gau_loss, sp_loss, poi_loss))

In [171]:
mn = 100
for ep in range(epoch):
    g,s,p,now = 0,0,0,0
    for batch_idx, (origin, gau, sp, poi) in enumerate(train_loader):
        origin, gau, sp, poi = origin.cuda(), gau.cuda(), sp.cuda(), poi.cuda()
        output1 = model_RGB(gau)
        gau_loss = criterion(output1, origin)
        output2 = model_RGB(sp)
        sp_loss = criterion(output2, origin)
        output3 = model_RGB(poi)
        poi_loss = criterion(output3, origin)
        loss_1, loss_2, loss_3 = criterion(output1, output2),criterion(output1, output3),criterion(output3, output2)
        loss = (gau_loss + sp_loss + poi_loss) + (loss_1+loss_2+loss_3)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
            
        g += gau_loss.item()
        s += sp_loss.item()
        p += poi_loss.item()

    #lr_scheduler.step()
    # l = len(train_set)
    # g /= l
    # s /= l
    # p /= l
    now = (g+s+p)/3
    if now < mn:
        mn = now
        save_checkpoint('/content/drive/MyDrive/torch/DIP/DenoiseAE_UNet_loss.pth', model, optimizer)

    print('Train Epoch: {} Gaussian loss: {:.4f}, Salt&Pepper loss: {:.4f}, Poisson loss: {:.4f}\n'.format(ep, g, s, p))
    test(model)

model saved to /content/drive/MyDrive/torch/DIP/DenoiseAE_UNet_loss.pth
Train Epoch: 0 Gaussian loss: 0.1032, Salt&Pepper loss: 0.0996, Poisson loss: 0.0782


Validation set: Gaussian loss: 0.0171, Salt&Pepper loss: 0.0147, Poisson loss: 0.0110

model saved to /content/drive/MyDrive/torch/DIP/DenoiseAE_UNet_loss.pth
Train Epoch: 1 Gaussian loss: 0.0650, Salt&Pepper loss: 0.0581, Poisson loss: 0.0431


Validation set: Gaussian loss: 0.0170, Salt&Pepper loss: 0.0147, Poisson loss: 0.0109

Train Epoch: 2 Gaussian loss: 0.0649, Salt&Pepper loss: 0.0581, Poisson loss: 0.0433


Validation set: Gaussian loss: 0.0169, Salt&Pepper loss: 0.0146, Poisson loss: 0.0110

Train Epoch: 3 Gaussian loss: 0.0648, Salt&Pepper loss: 0.0581, Poisson loss: 0.0436


Validation set: Gaussian loss: 0.0169, Salt&Pepper loss: 0.0146, Poisson loss: 0.0111

model saved to /content/drive/MyDrive/torch/DIP/DenoiseAE_UNet_loss.pth
Train Epoch: 4 Gaussian loss: 0.0643, Salt&Pepper loss: 0.0577, Poisson loss: 0.0435


V

KeyboardInterrupt: ignored

# Evaluate

In [205]:
#model = DAE().cuda()
model.eval()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()
load_checkpoint('/content/drive/MyDrive/torch/DIP/DenoiseAE_UNet_loss.pth', model, optimizer)

t2i = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(512),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
i2t = transforms.Compose([
    transforms.ToPILImage(),
    #transforms.Resize((720,720)),
])

stn = Image.open('blurred/402.png')
stn = t2i(stn).cuda()
img = Image.open('blurred_noise/402_poisson.png')
im_t = t2i(img).cuda()
# stn = train_set[305][0].cuda()
# im_t = train_set[305][1].cuda()
im_o2 = i2t(im_t)
print(im_t.shape)
stn_t = torch.unsqueeze(stn,0)
im_t = torch.unsqueeze(im_t,0)
# im_t = im_t.unfold(2, 64, 64).unfold(3, 64, 64)
# output = torch.zeros(1,3,768,768)
# for i in range(12):
#     for j in range(12):
#         output[:,:,i*64:(i+1)*64,j*64:(j+1)*64] = model(im_t[:,:,i,j,:,:])
#         #output[:,:,i*64:(i+1)*64,j*64:(j+1)*64] = im_t[:,:,i,j,:,:]

# for i in range(5):
#     for j in range(5):
#         im_t[:,:,i*128:(i+1)*128,j*128:(j+1)*128] = model(im_t[:,:,i*128:(i+1)*128,j*128:(j+1)*128])
im_t = model_RGB(im_t)

loss = criterion(stn_t,im_t)
stn_t = torch.squeeze(stn_t,0)
stn = i2t(stn_t)
im_t = torch.squeeze(im_t,0)
im_o = i2t(im_t)

model loaded from /content/drive/MyDrive/torch/DIP/DenoiseAE_UNet_loss.pth
torch.Size([3, 720, 720])


In [206]:
im_o.save('denoise.png')
im_o2.save('noise.png')
stn.save('blurred.png')

In [188]:
loss

tensor(0.0005, device='cuda:0', grad_fn=<MseLossBackward0>)